In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import math

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
customer = pd.read_pickle("../new_data/data_set/customer.pkl")
online = pd.read_pickle("../new_data/data_set/online.pkl")
product = pd.read_pickle("../new_data/data_set/product.pkl")
transaction = pd.read_pickle("../new_data/data_set/transaction.pkl")
clac = pd.read_pickle("../new_data/data_set/online_clac.pkl")
group = pd.read_pickle("../new_data/data_set/online_group.pkl")

# Transaction 

In [ ]:
buy = transaction.loc[transaction['biz_unit']=='A']
buy[['buy_am','buy_ct']] = buy[['buy_am','buy_ct']].astype(float)
buy_ct = buy[['clnt_id','buy_ct']].groupby('clnt_id').sum()
buy_am = buy[['clnt_id','buy_am']].groupby('clnt_id').sum()
tran_freq = buy[['clnt_id','trans_id']]
tran_freq = tran_freq.drop_duplicates(['clnt_id','trans_id'],keep='first')
tran_freq['tran_freq'] = 1
tran_freq = tran_freq[['clnt_id','tran_freq']]
tran_freq = tran_freq.groupby('clnt_id').sum()
tran_ct = buy[['clnt_id','trans_id','buy_ct']].groupby(['clnt_id','trans_id']).sum()
tran_ct = tran_ct.groupby('clnt_id').mean()
tran_ct.columns = ['tran_ct']
tran_am = buy[['clnt_id','trans_id','buy_am']].groupby(['clnt_id','trans_id']).sum()
tran_am = tran_am.groupby('clnt_id').mean()
tran_am.columns = ['tran_am']

In [ ]:
clac_var = (clac != 0).astype(int)
clac_var = pd.DataFrame(clac_var.sum(axis=1))
clac_var.columns = ['clac_var']
clac_ct = pd.merge(buy_ct,clac_var,on='clnt_id',how='left')
clac_ct['clac_ct'] = clac_ct['buy_ct']/clac_ct['clac_var']
clac_ct = pd.DataFrame(clac_ct['clac_ct'])
clac_am = pd.merge(buy_am,clac_var,on='clnt_id',how='left')
clac_am['clac_am'] = clac_am['buy_am']/clac_am['clac_var']
clac_am = pd.DataFrame(clac_am['clac_am'])

In [ ]:
basic = pd.merge(buy_ct,buy_am,on='clnt_id',how='left')
basic = pd.merge(basic,tran_freq,on='clnt_id',how='left')
basic = pd.merge(basic,tran_ct,on='clnt_id',how='left')
basic = pd.merge(basic,tran_am,on='clnt_id',how='left')
basic = pd.merge(basic,clac_var,on='clnt_id',how='left')
basic = pd.merge(basic,clac_ct,on='clnt_id',how='left')
basic = pd.merge(basic,clac_am,on='clnt_id',how='left')

In [ ]:
basic.to_pickle("../new_data/online/basic.pkl")

# Time 

## 1) week 

In [ ]:
tran_time = transaction.loc[transaction['biz_unit']=='A']
tran_time['month'] = np.array([i.month for i in tran_time['date_time']])
tran_time['day'] = np.array([i.day for i in tran_time['date_time']])
tran_time['time'] = np.array([i.hour for i in tran_time['date_time']])
tran_time['weekday'] = np.array([i.weekday() for i in tran_time['date_time']])

In [ ]:
weekday = tran_time[['clnt_id','pd_c','weekday']]
weekday['count'] = 1
weekday = pd.pivot_table(weekday, index='clnt_id',columns='weekday',values='count',aggfunc=sum)
weekday = weekday.fillna(0)
weekday.columns = ['mon','tue','wed','thu','fri','sat','sun']

In [ ]:
week_plot = pd.melt(weekday,var_name='days',value_name='count')
sns.boxplot(x='days',y='count',data=week_plot, showfliers=False)
plt.title('distribution of transaction by weekdays')
plt.show()

In [ ]:
week = weekday[['mon','tue','wed','thu','fri']]
week = pd.DataFrame(week.sum(axis=1))
weekend = weekday[['sat','sun']]
weekend = pd.DataFrame(weekend.sum(axis=1))
weekday = pd.merge(week,weekend,on='clnt_id',how='outer')
weekday.columns = ['week','weekend']

In [ ]:
weekday['week'] = weekday['week']/5
weekday['weekend'] = weekday['weekend']/2

In [ ]:
week_plot = pd.melt(weekday,var_name='days',value_name='count')
sns.boxplot(x='days',y='count',data=week_plot, showfliers=False)
plt.title('distribution of transaction by week / weekend')
plt.show()

## 2) time 

In [ ]:
time = tran_time[['clnt_id','pd_c','time']]
time['count'] = 1
time = pd.pivot_table(time, index='clnt_id',columns='time',values='count',aggfunc=sum)
time = time.fillna(0)

In [ ]:
time_plot = pd.melt(time,var_name='time',value_name='count')
sns.boxplot(x='time',y='count',data=time_plot.loc[time_plot['count']!=0], showfliers=False)
plt.title('distribution of transaction by time')
plt.show()

In [ ]:
tran_time['timezone'] = 0
tran_time['timezone'].loc[[i in [0,1,2,3,4,5,6,7,8] for i in tran_time['time']]] = 1
tran_time['timezone'].loc[[i in [9,10,11,12,13,14,15,16] for i in tran_time['time']]] = 2
tran_time['timezone'].loc[[i in [17,18,19,20,21,22,23] for i in tran_time['time']]] = 3
# tran_time['timezone'].loc[[i in [15,16,17,18,19,20] for i in tran_time['time']]] = 4

In [ ]:
time = tran_time[['clnt_id','pd_c','timezone']]
time['count'] = 1
time = pd.pivot_table(time, index='clnt_id',columns='timezone',values='count',aggfunc=sum)
time = time.fillna(0)
time.columns = ['time'+str(i+1) for i in range(3)]

In [ ]:
time['time1'] = time['time1']/9
time['time2'] = time['time2']/8
time['time3'] = time['time3']/7
# time['time4'] = time['time4']/6

In [ ]:
time_plot = pd.melt(time,var_name='timezone',value_name='count')
sns.boxplot(x='timezone',y='count',data=time_plot.loc[time_plot['count']!=0], showfliers=False)
plt.title('distribution of transaction by timezone')
plt.show()

In [ ]:
time = pd.merge(weekday,time,on='clnt_id',how='outer')
time.to_pickle("../new_data/online/time.pkl")

# Online

In [ ]:
# customer information
online_customer = pd.DataFrame(online['clnt_id'].unique())
online_customer.columns = ['clnt_id']

In [ ]:
# transaction proportion
online_tran = online[['clnt_id','sess_id','action_type']]
online_tran['transaction'] = (online_tran['action_type']=='6').astype(int)
online_tran = online_tran.drop('action_type',axis=1)
online_tran = online_tran.groupby(['clnt_id','sess_id']).sum()
online_tran = (online_tran > 0).astype(int)
online_tran = online_tran.groupby('clnt_id').mean()

In [ ]:
# search information
online_search = online[['clnt_id','action_type']]
online_search = online_search.loc[online_search['action_type']=='0']
online_search = pd.DataFrame(online_search.groupby('clnt_id').size())
# basket information
online_basket = online[['clnt_id','action_type']]
online_basket = online_basket.loc[online_basket['action_type']=='3']
online_basket = pd.DataFrame(online_basket.groupby('clnt_id').size())

In [ ]:
online_trfc = online[['clnt_id','sess_id','trfc_src']]
online_trfc = online_trfc.drop_duplicates(['clnt_id','sess_id'],keep='first')
online_trfc['count'] = 1
online_trfc = pd.pivot_table(online_trfc,index='clnt_id',columns='trfc_src',values='count',aggfunc=sum)
online_trfc = online_trfc.fillna(0)
trfc_plot = pd.melt(online_trfc,var_name='trfc_src',value_name='count')
sns.boxplot(x='trfc_src',y='count',data=trfc_plot)
plt.show()

In [ ]:
online_trfc['portal'] = online_trfc[['PORTAL_1','PORTAL_2','PORTAL_3']].sum(axis=1)
online_trfc = online_trfc.drop(['PORTAL_1','PORTAL_2','PORTAL_3'],axis=1)

In [ ]:
# dvc information
online_dvc = online[['clnt_id','sess_id','dvc_ctg_nm']]
online_dvc = online_dvc.drop_duplicates(['clnt_id','sess_id'],keep='first')
online_dvc['count'] = 1
online_dvc = pd.pivot_table(online_dvc,index='clnt_id',columns='dvc_ctg_nm',values='count',aggfunc=sum)
online_dvc = online_dvc.fillna(0)

In [ ]:
# sesstion view information
# 한 번의 구매당 평균 view 횟수
# 소비마다 구매 양상이 다르지 않을 것이다.(ex, 신중히 구매하는 타입, 대충 구매하는 타입 등등)
# online_buy = online.loc[online['action_type']=='6']
online_view = online[['clnt_id','tot_pag_view_ct']]
online_view = online_view.fillna(0)
online_view['tot_pag_view_ct'] = online_view['tot_pag_view_ct'].astype(int)
online_view = pd.DataFrame(online_view.groupby('clnt_id').mean())

In [ ]:
# sesstion hour information
# 한 번의 구매당 평균 sesstion 시간
# online_buy = online.loc[online['action_type']=='6']
online_hour = online[['clnt_id','tot_sess_hr_v']]
online_hour = online_hour.fillna(0)
online_hour['tot_sess_hr_v'] = online_hour['tot_sess_hr_v'].astype(int)
online_hour = pd.DataFrame(online_hour.groupby('clnt_id').mean())

In [ ]:
# merge information
online_total = pd.merge(online_customer,online_tran,on='clnt_id',how='left')
online_total = pd.merge(online_total,online_search,on='clnt_id',how='left')
online_total = pd.merge(online_total,online_basket,on='clnt_id',how='left')
#online_total = pd.merge(online_total,online_trfc,on='clnt_id',how='left')
#online_total = pd.merge(online_total,online_dvc,on='clnt_id',how='left')
online_total = pd.merge(online_total,online_view,on='clnt_id',how='left')
online_total = pd.merge(online_total,online_hour,on='clnt_id',how='left')
online_total.columns = ['clnt_id','transaction','search','basket','view','hour']
online_total = online_total.fillna(0)
online_total = online_total.set_index('clnt_id')

In [ ]:
online_total

In [ ]:
online_total.to_pickle("../new_data/online/online.pkl")

# Group

In [ ]:
on_tran = transaction.loc[transaction['biz_unit']=='A']
on_tran['pd_c'] = on_tran['pd_c'].astype(int)
group['pd_c'] = group['pd_c'].astype(int)

In [ ]:
on_group = pd.merge(on_tran[['clnt_id','pd_c']],group[['pd_c','group']],on='pd_c',how='left')
on_group['count'] = 1
on_group = pd.pivot_table(on_group,index='clnt_id',columns='group',values='count',aggfunc=sum)
on_group = on_group.fillna(0)

In [ ]:
on_group.to_pickle("../new_data/online/ongroup.pkl")